In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:

spark = SparkSession.builder.appName('flights').getOrCreate()

22/12/21 19:28:50 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.156 instead (on interface en0)
22/12/21 19:28:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 19:28:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def count_null(df):
    cols_check = df.columns
    df.select(*[
    (
        F.count(F.when((F.isnan(c) | F.col(c).isNull()), c)) if t not in ("timestamp", "date")
        else F.count(F.when(F.col(c).isNull(), c))
    ).alias(c)
    for c, t in df.dtypes if c in cols_check
]).show()

In [4]:
def select_features():
    features = []
    with open("features_analysis.txt","r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line=="":
                continue
            split = line.split()
            feature = split[0].strip()
            features.append(feature)
    return features


In [5]:
def clean_data(df,features):
    df = df.select(features)
    df = df.na.drop()
    return df

In [6]:
def join_airports(df,airports):
    df = df.join(airports,df.Origin==airports.IATA)
    df = df.withColumnRenamed("LATITUDE","ORIGIN_LATITUDE")
    df = df.withColumnRenamed("LONGITUDE","ORIGIN_LONGITUDE")
    df = df.withColumnRenamed("STATE","ORIGIN_STATE")
    df = df.drop("IATA","AIRPORT","CITY","COUNTRY")
    df = df.join(airports,df.Dest==airports.IATA)
    df = df.withColumnRenamed("LATITUDE","DEST_LATITUDE")
    df = df.withColumnRenamed("LONGITUDE","DEST_LONGITUDE")
    df = df.withColumnRenamed("STATE","DEST_STATE")
    df = df.drop("IATA","AIRPORT","CITY","COUNTRY")
    return df

In [3]:
# get all files with .csv extension
import glob
files = glob.glob("../data.nosync/*.csv")

In [4]:
df_tmp = spark.read.csv(files[0],header=True,inferSchema=True)

In [10]:
schema = df_tmp.schema

In [6]:
# save schema as json
with open("schema.json","w") as f:
    f.write(schema.json())


In [9]:
# load schema from json
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
with open("schema.json","r") as f:
    schema_loaded = StructType.fromJson(json.load(f))


In [10]:
def preprocess_normal_data(path):
    df = spark.read.csv(path, schema=schema, header=True)
    columns = df.columns
    df_normal = df.filter(df['Diverted'] == 0)
    features = select_features()
    df_normal = clean_data(df_normal,features)

    airports = spark.read.csv("airports.csv", inferSchema=True, header=True)
    df_normal = join_airports(df_normal,airports)
    return df_normal

In [11]:
df_cleaned = preprocess_normal_data("../data.nosync")


In [12]:
df_cleaned.count()

22/12/21 13:28:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


6246739

In [13]:
# save the cleaned data
df_cleaned.write.csv("../data.nosync/cleaned/cleaned_flights.csv",header=True)